In [1]:
from beir import util, LoggingHandler
from beir.retrieval import models
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES

import logging
import pathlib, os

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])


from torchdr import PCA, TSNE
from sentence_transformers import SentenceTransformer
import pandas as pd


/home/dijkstraz/anaconda3/envs/colab/lib/python3.10/site-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
/home/dijkstraz/anaconda3/envs/colab/lib/python3.10/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [ ]:
model_name_list =[
    # 'all-mpnet-base-v2', #  0.5481, 0.2312, 0.2559 (1 mins) (2312, 2559)
    # # 'all-mpnet-base-v2', #[full 10] 2414 -> 2684kwo, 2440kw, 2298kp, 2740kpo, 2352kso, 2399ks
    # 'sentence-t5-xl', #[full] 0.6754, 0.2543, 0.2990

    'all-MiniLM-L12-v1',
    'all-mpnet-base-v2',
]

reduction_classes = {
    'pca': PCA,
    # 'tsne': TSNE
}

class idenity_reduction:

    def __init__(self, *args, **kwargs):
        pass

    def fit(self, x):
        pass

    def transform(self, x):
        return x

class ST_wrapper(SentenceTransformer):
    def __init__(self, model_name, reduction_type = 'x', reduction_kwargs={}, *args, **kwargs):
        super(ST_wrapper, self).__init__(model_name, *args, **kwargs)
        if reduction_type == 'x':
            self.reduction = idenity_reduction()
        else:
            self.reduction = reduction_classes[reduction_type](**reduction_kwargs)

    def encode_queries(self, queries, *args, **kwargs):
        embeddings = self.encode(queries, *args, **kwargs)
        self.reduction.fit(embeddings)
        return self.reduction.transform(embeddings)

    def encode_corpus(self, corpus, *args, **kwargs):
        embeddings = self.encode(corpus, *args, **kwargs)
        print(embeddings.shape)
        return self.reduction.transform(embeddings)


reduction_kwargs_choices = {
    'x': {},
    'pca': {'n_components': 128},
    # 'tsne': {'perplexity': 30}
}

results_dict = {}

out_dir = "./beir"
os.mkdir(out_dir) if not os.path.exists(out_dir) else None
#### /print debug information to stdout

#### Download scifact.zip dataset and unzip the dataset
dataset_list = ['scifact', 'scidocs', 'climate-fever', 'hotpotqa']

df_list = []
for dataset in dataset_list:

    # dataset = "scidocs"
    url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)

    data_path = util.download_and_unzip(url, out_dir)

    #### Provide the data_path where scifact has been downloaded and unzipped
    corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")


    for model_name in model_name_list:
        for reduction_type in ['pca', 'x', ]:

            model = ST_wrapper(model_name, reduction_type, reduction_kwargs_choices[reduction_type])
            model = DRES(model, batch_size=128)

            retriever = EvaluateRetrieval(model, score_function="dot") # or "cos_sim" for cosine similarity
            results = retriever.retrieve(corpus, queries)
            ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)
            results_dict[(model_name, reduction_type)] = {}
            for result in [ndcg, _map, recall, precision]:
                for k, v in result.items():
                    results_dict[(model_name, reduction_type)][k] = v

        df = pd.DataFrame(results_dict)
        df_list.append(df)

2025-02-03 02:45:15 - Loading Corpus...


  0%|          | 0/5183 [00:00<?, ?it/s]

2025-02-03 02:45:15 - Loaded 5183 TEST Documents.
2025-02-03 02:45:15 - Doc Example: {'text': 'Alterations of the architecture of cerebral white matter in the developing human brain can affect cortical development and result in functional disabilities. A line scan diffusion-weighted magnetic resonance imaging (MRI) sequence with diffusion tensor analysis was applied to measure the apparent diffusion coefficient, to calculate relative anisotropy, and to delineate three-dimensional fiber architecture in cerebral white matter in preterm (n = 17) and full-term infants (n = 7). To assess effects of prematurity on cerebral white matter development, early gestation preterm infants (n = 10) were studied a second time at term. In the central white matter the mean apparent diffusion coefficient at 28 wk was high, 1.8 microm2/ms, and decreased toward term to 1.2 microm2/ms. In the posterior limb of the internal capsule, the mean apparent diffusion coefficients at both times were similar (1.2 vers

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

2025-02-03 02:45:17 - Sorting Corpus by document length (Longest first)...
2025-02-03 02:45:17 - Encoding Corpus in batches... Warning: This might take a while!
2025-02-03 02:45:17 - Scoring Function: Dot Product (dot)
2025-02-03 02:45:17 - Encoding Batch 1/1...


...

  0%|          | 0/25657 [00:00<?, ?it/s]

2025-02-03 02:46:03 - Loaded 25657 TEST Documents.
2025-02-03 02:46:03 - Doc Example: {'text': 'An evolutionary recurrent network which automates the design of recurrent neural/fuzzy networks using a new evolutionary learning algorithm is proposed in this paper. This new evolutionary learning algorithm is based on a hybrid of genetic algorithm (GA) and particle swarm optimization (PSO), and is thus called HGAPSO. In HGAPSO, individuals in a new generation are created, not only by crossover and mutation operation as in GA, but also by PSO. The concept of elite strategy is adopted in HGAPSO, where the upper-half of the best-performing individuals in a population are regarded as elites. However, instead of being reproduced directly to the next generation, these elites are first enhanced. The group constituted by the elites is regarded as a swarm, and each elite corresponds to a particle within it. In this regard, the elites are enhanced by PSO, an operation which mimics the maturing pheno

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

2025-02-03 02:46:04 - Sorting Corpus by document length (Longest first)...
2025-02-03 02:46:04 - Encoding Corpus in batches... Warning: This might take a while!
2025-02-03 02:46:04 - Scoring Function: Dot Product (dot)
2025-02-03 02:46:04 - Encoding Batch 1/1...


Batches:   0%|          | 0/201 [00:00<?, ?it/s]

In [5]:
pd.concat(df_list)

all-MiniLM-L12-v1         
                          pca      NaN
NDCG@1                0.38667  0.49667
NDCG@3                0.46472  0.56936
NDCG@5                0.48924  0.59417
NDCG@10               0.51689  0.62170
NDCG@100              0.56530  0.65990
NDCG@1000             0.57961  0.66767
MAP@1                 0.36872  0.48139
MAP@3                 0.43694  0.54461
MAP@5                 0.45279  0.56156
MAP@10                0.46607  0.57537
MAP@100               0.47659  0.58381
MAP@1000              0.47709  0.58409
Recall@1              0.36872  0.48139
Recall@3              0.51856  0.61589
Recall@5              0.58006  0.67694
Recall@10             0.65944  0.75400
Recall@100            0.88267  0.93000
Recall@1000           0.99667  0.99167
P@1                   0.38667  0.49667
P@3                   0.18778  0.22333
P@5                   0.12800  0.15067
P@10                  0.07500  0.08567
P@100                 0.01003  0.01057
P@1000                0.00113  0.00112